In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from cycler import cycler
from IPython import display
import datetime

plt.rcParams['axes.facecolor'] = '#0057b8' # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv', index_col='row_id', parse_dates=['time'])
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv', index_col='row_id', parse_dates=['time'])

print(train.shape, test.shape)
train.head()

In [ ]:


# Unique roadways
roadways = train[['x', 'y']].drop_duplicates()
display.display(roadways)

# Unique directions
print('Unique directions:', train.direction.unique())

# Unique roadways with direction
road_dir = train[['x', 'y', 'direction']].drop_duplicates()
display.display(road_dir)



In [ ]:
#12 roadways, 8 directions and 65 combinations of roadway with direction

In [ ]:
#plot a geographical map of the 12 places with the directions
dir_dict = {'EB': (1, 0), 'NB': (0, 1), 'SB': (0, -1), 'WB': (-1, 0), 'NE': (1, 1), 'SE': (-1, 1), 'NW': (1, -1), 'SW': (-1, -1)}

plt.figure(figsize=(8, 8))
plt.scatter(roadways.x, roadways.y)
plt.gca().set_aspect('equal')
for _, x, y, d in road_dir.itertuples():
    dx, dy = dir_dict[d]
    dx, dy = dx/4, dy/4
    plt.plot([x, x+dx], [y, y+dy], color='#ffd700')
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True)) # only integer labels
plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True)) # only integer labels
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
#checking whether (0,0) has 3 direction
road_dir[(road_dir['x'] == 0) & (road_dir['y'] ==0 )]

In [ ]:
#checking whether (1,0) has 6 direction
road_dir[(road_dir['x'] == 1) & (road_dir['y'] ==0 )]

In [ ]:
unique_times = train.time.unique()
print(unique_times.shape)

In [ ]:
train.shape

In [ ]:
print(unique_times.min(), unique_times.max())
print(test.time.min(), test.time.max())
print(np.diff(unique_times)[0])
unique_times[1:][np.diff(unique_times) != np.diff(unique_times)[0]]

In [ ]:
#note : timestamps are between 1991-04-01 00:00 and 1991-09-30 11:40


In [ ]:
np.unique(train.congestion)

In [ ]:
plt.figure(figsize=(10, 6))
#plt.hist(train.congestion, bins=np.linspace(-0.5, 100.5, 102))
plt.bar(range(101), train.congestion.value_counts().sort_index(), width=1,
        color=['r' if con in [15, 20, 21, 29, 34] else '#ffd700' for con in range(101)])
plt.ylabel('Count')
plt.xlabel('Congestion')
plt.show()

In [ ]:
plt.subplots(2, 2, sharex=True, sharey=True, figsize=(16, 12))
for y in range(4):
    plt.subplot(2, 2, y+1)
    vc = train[(train.x == 2) & (train.y == y)].congestion.value_counts().sort_index()
    plt.bar(vc.index, vc, width=1,
            color=['r' if con in [15, 20, 21, 29, 34] else '#ffd700' for con in vc.index])
    plt.ylabel('Count')
    plt.xlabel('Congestion')
    plt.title(f"(x = {2}) & (y = {y})")
plt.show()

In [ ]:
for direction in train.direction.unique():
    temp = train[(train.x == 2) & (train.y == 1) & (train.direction == direction)]
    plt.subplots(1, 2, figsize=(18, 4))
    plt.subplot(1, 2, 1)
    vc = temp.congestion.value_counts().sort_index()
    plt.bar(vc.index, vc, width=1,
            color=['r' if con in [15, 20, 21, 29, 34] else '#ffd700' for con in vc.index])
    plt.ylabel('Count')
    plt.xlabel('Congestion')
    plt.title(f"(x = {2}) & (y = {1}) & (direction = {direction})")
    plt.subplot(1, 2, 2)
    plt.scatter(temp.time, temp.congestion, s=1, color=['r' if con in [15, 20, 21, 29, 34] else '#ffd700' for con in temp.congestion])
    plt.title(f"(x = {2}) & (y = {1}) & (direction = {direction})")
    plt.ylabel('Congestion')
    plt.show()

In [ ]:
#Dependence on time and date
temp = train.groupby(train.time.dt.dayofweek).congestion.mean()
plt.figure(figsize=(18, 6))
plt.title('Days of the week')
plt.bar(temp.index, temp, color='#ffd700')
plt.xticks(ticks=temp.index, labels='MTWTFSS')
plt.show()

In [ ]:
temp = train.groupby(train.time.dt.hour + train.time.dt.minute/60).congestion.mean()
plt.figure(figsize=(18, 6))
plt.title('Time of the day')
plt.bar(temp.index, temp, color='#ffd700', width=0.34)
plt.xticks(range(24))
plt.xlabel('Time of day')
plt.ylabel('Congestion')
plt.show()

In [ ]:
# the congestion higher peak in the late afternoon

In [ ]:


plt.subplots(2, 2, sharex=True, sharey=True, figsize=(16, 12))
for y in range(4):
    plt.subplot(2, 2, y+1)
    vc = train[(train.x == 2) & (train.y == y)]
    temp = vc.groupby(vc.time.dt.hour + vc.time.dt.minute/60).congestion.mean()
    plt.bar(temp.index, temp, color='#ffd700', width=0.34)
    plt.xticks(range(24))
    plt.xlabel('Time of day')
    plt.ylabel('Congestion')
    plt.title(f"(x = {2}) & (y = {y})")
plt.show()



In [ ]:
#every road has its own pattern

In [ ]:
#Trend


temp = train.groupby(train.time.dt.date).congestion.mean()
plt.figure(figsize=(18, 6))
plt.title('Trend')
poly = np.polynomial.polynomial.Polynomial.fit(range(len(temp.index)), temp, deg=1)
plt.plot(temp.index, poly(range(len(temp.index))))
plt.ylim(47, 48)
plt.show()

